In [ ]:
import numpy as np
import pandas as pd
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.survival.tsv
!wget https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/Survival_SupplementalTable_S1_20171025_xena_sp
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.basic_phenotype.tsv.gz
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.methylation27.tsv.gz
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.htseq_fpkm-uq.tsv.gz

In [ ]:
surv = pd.read_csv('Survival_SupplementalTable_S1_20171025_xena_sp', sep='\t')
meta_df = pd.read_csv('GDC-PANCAN.basic_phenotype.tsv.gz', sep='\t')
meth_cols = pd.read_csv('GDC-PANCAN.methylation27.tsv.gz', compression='gzip', sep='\t',nrows=0).columns
ge_cols = pd.read_csv('GDC-PANCAN.htseq_fpkm-uq.tsv.gz',compression='gzip', sep='\t', index_col=0, nrows=0).columns
surv_gdc = pd.read_csv('GDC-PANCAN.survival.tsv' , sep='\t')
surv_gdc = surv_gdc.set_index('sample')
print(surv_gdc.shape)
print(meta_df.shape)
meta_df = meta_df.set_index('sample')
## Metafile analysis
## Meta file analysis and label dictionary creation
c_list=[]
rs_list=[]
key_control = ['Blood Derived Normal','Bone Marrow Normal','Buccal Cell Normal','FFPE Scrolls','Solid Tissue Normal' ]
key_case = ['Additional - New Primary', 'Additional Metastatic','Human Tumor Original Cells','Metastatic','Primary Blood Derived Cancer - Bone Marrow','Primary Blood Derived Cancer - Peripheral Blood','Primary Tumor','Recurrent Blood Derived Cancer - Bone Marrow','Recurrent Blood Derived Cancer - Peripheral Blood','Recurrent Tumor']
meta_df_control = meta_df[meta_df['sample_type'].isin(key_control)]
meta_df_case = meta_df[meta_df['sample_type'].isin(key_case)]
label_dict = {}
s1 = list(set(meta_df_control.index) & set(ge_cols) & set(meth_cols))
pan_key_normal = []
pan_key_cancer = []
for k in s1:
  label_dict[k] = "C"
s2 = list(set(meta_df_case.index) &set(ge_cols) & set(meth_cols))
for k in s2:
  label_dict[k] = "RS"

## Finding normal and tumor samples in different cancers
##  dis_list --> s_tcga : [{key1:value1,key2:value2}]
s_tcga = {}
dis_list = meta_df['project_id'].unique()
for dis in dis_list:
  key_normal = []
  key_nan = []
  key_cancer = []
  meta_df1 = meta_df[meta_df['project_id'] == dis]
  meta_df1.reset_index(inplace = True)
  s_tcga_i ={}
  for i in range(0,meta_df1.shape[0]):
    if meta_df1['sample_type'][i] in key_control:
      key_normal.append(meta_df1['sample'][i])
      pan_key_normal.append(meta_df1['sample'][i])
    elif pd.isnull(meta_df1['sample_type'][i]):
      key_nan.append(meta_df1['sample'][i])
    else:
      key_cancer.append(meta_df1['sample'][i])
      pan_key_cancer.append(meta_df1['sample'][i])

  samples_to_use_i = []
  samples_to_use_i = key_normal + key_cancer
  key_normal = set(ge_cols) & set(meth_cols) & set(key_normal)
  key_normal = list(key_normal)
  key_cancer =  set(ge_cols) & set(meth_cols) & set(key_cancer)
  key_cancer = list(key_cancer)

  pan_key_normal = set(ge_cols) & set(meth_cols) & set(pan_key_normal)
  pan_key_normal = list(pan_key_normal)
  pan_key_cancer =  set(ge_cols) & set(meth_cols) & set(pan_key_cancer)
  pan_key_cancer = list(pan_key_cancer)
  s_tcga_i['normal'] = key_normal
  s_tcga_i['cancer'] = key_cancer
  if len(samples_to_use_i) != 0 :
    s_tcga[dis] = s_tcga_i

  C_count = len(key_normal)
  c_list.append(C_count)
  print(f"Control Samples :{C_count}")
  RS_count = len(key_cancer)
  rs_list.append(RS_count)
  print(f"Tumour Samples :{RS_count}")
C_count = len(pan_key_normal)
print(f"Pan Control Samples :{C_count}")
RS_count = len(pan_key_cancer)
print(f"Pan Tumour Samples :{RS_count}")
c_dict = {
  "disease": dis_list,
  "cancer_samples": rs_list,
  "normal samples": c_list
}
c_df = pd.DataFrame(c_dict)
print(c_df)


In [ ]:
## Sample selection
samples_to_use = pan_key_normal + pan_key_cancer
samples_to_use = list(samples_to_use)
print("Sample Size")
print(len(samples_to_use))

In [ ]:
!pip install pyDESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import VarianceThreshold

In [ ]:
## DNA Methylation Data
## Reading and Preprocessing....
samples_to_use.append(('Composite Element REF'))
meth = pd.read_csv('GDC-PANCAN.methylation27.tsv.gz', compression='gzip', sep='\t' ,usecols =samples_to_use)
print('Meth Sample size:')
print(len(meth.columns))
print(meth.shape)
meth = meth.dropna(axis=0)
print(meth.shape)
meth.set_index("Composite Element REF", inplace = True)
meth_df = meth.T
print(meth_df.shape)
var_thr = VarianceThreshold(threshold = 0.001) #Removing features that are 95% or more similar
var_thr.fit(meth_df)
concol = [column for column in meth_df.columns
          if column not in meth_df.columns[var_thr.get_support()]]
meth_df = meth_df.drop(concol,axis=1)
print("Meth shape after deleting less variance features")
print(meth_df.shape)
##Differential Analysis
Lab = [label_dict[i] for i in meth_df.index]
meth_dff= (meth_df * 1000).astype(int)
metadata = pd.DataFrame(zip(meth_dff.index,Lab),columns=('Sample','Condition'))
print(metadata.groupby('Condition')['Sample'].nunique())
metadata=metadata.set_index('Sample')
dds = DeseqDataSet(counts = meth_dff, metadata = metadata, design_factors = 'Condition')
print(dds)
dds.deseq2()
stat_res = DeseqStats(dds, contrast = ('Condition','RS','C'))
print(stat_res.summary())
res = stat_res.results_df
m = res[(res.padj < 0.05) & (abs(res.log2FoldChange)>0.5)]
print("Significant Genes")
print(m)
hyper = m[m['log2FoldChange']>0]
hypo = m[m['log2FoldChange']<0]
hypo_cpg = list(hypo.index)
hyper_cpg = list(hyper.index)
print("Hyper-length")
print(len(hyper_cpg))
print("Hypo-length")
print(len(hypo_cpg))


In [ ]:
## Gene Expression Data
## Reading and Preprocessing....
samples_to_use.append('xena_sample')
ge = pd.read_csv('GDC-PANCAN.htseq_fpkm-uq.tsv.gz', compression='gzip', index_col=0, sep ='\t',usecols =samples_to_use)
ge = ge.T
print(ge.shape)
var_thr = VarianceThreshold(threshold = 0.001)
var_thr.fit(ge)
concol = [column for column in ge.columns
          if column not in ge.columns[var_thr.get_support()]]
ge_df = ge.drop(concol,axis=1)
print("Meth shape after deleting less variance features")
print(ge_df.shape)
drop_col=[]
for i in ge_df.columns:
  count = (ge_df[i] == 0).sum()
  if count>(ge_df.shape[0]*0.50):
    drop_col.append(i)
print(len(drop_col))
ge_df=ge_df.drop(columns=drop_col)
print("GE shape after deleting zero columns:")
print(ge_df.shape)
ge_df = ge_df.dropna(axis=1)
print("GE shape after deleting na")
print(ge_df.shape)
## Differential Analysis
Labels = [label_dict[y] for y in ge_df.index]
metadataG = pd.DataFrame(zip(ge_df.index,Labels),columns=('Sample','Condition'))
print(metadataG.groupby('Condition')['Sample'].nunique())
metadataG=metadataG.set_index('Sample')
ge1 =(ge_df).astype(int)
dds = DeseqDataSet(counts = ge1, metadata = metadataG, design_factors = 'Condition')
print(dds)
dds.deseq2()
stat_res = DeseqStats(dds, contrast = ('Condition','RS','C'))
print(stat_res.summary())
res = stat_res.results_df
g = res[(res.padj < 0.05) & (abs(res.log2FoldChange)>0.5)]
print("Significant Genes")
print(g)
up = g[g['log2FoldChange']>0]
down = g[g['log2FoldChange']<0]
up_id = list(up.index)
down_id = list(down.index)
print("Up-length")
print(len(up_id))
print("Down-length")
print(len(down_id))


In [ ]:
### Creating dictionaries
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/gencode.v22.annotation.gene.probeMap
!wget https://gdc-hub.s3.us-east-1.amazonaws.com/download/illuminaMethyl27_hg38_GDC
map={}
meth_map = pd.read_csv('illuminaMethyl27_hg38_GDC', sep='\t')
map_dict_m = dict(zip(meth_map['#id'], meth_map['gene']))
map_mm = pd.read_csv('cpg_methylation_cpg_to_annotation.tsv', sep='\t')
map_dict_mm = dict(zip(map_mm['CpG_id'], map_mm['Symbol']))
for (i,j) in map_dict_mm.items():
  if pd.isnull(j) or j=='.':
    map[i]=map_dict_m[i]
  else:
    k = map_dict_m[i]
    k_new = list(k.split(','))
    h = map_dict_mm[i]
    h_new = list(h.split(','))
    unique_values = set(k_new + h_new)
    if '.' in unique_values:
      unique_values.remove('.')
    map[i]=unique_values


In [ ]:
g_s_me=[]
to_drop=[]
cpg_to_gs={}
gs_to_cpg={}
gs_to_cpgg={}
mindex = list(m.index)
gindex = list(g.index)
for i in m.index:
  update = map[i]
  cpg_to_gs[i] = update
  for e in update:
    g_s_me.append(e)
    if e in gs_to_cpg.keys():
      gs_to_cpg[e]= gs_to_cpg[e] +','+ i
    else:
      gs_to_cpg[e] = i
gs_to_gid={}
gid_to_gs={}
g_s_ge=[]
ge_map = pd.read_csv('gencode.v22.annotation.gene.probeMap', sep='\t')
map_dict_r = dict(zip(ge_map['gene'], ge_map['id']))
map_dict = dict(zip(ge_map['id'], ge_map['gene']))
for i in gindex:
  gid_to_gs[i] = map_dict[i]
  gs_to_gid[map_dict[i]]=i
  g_s_ge.append(map_dict[i])
## Overlapping...
overlap_g_s = set(g_s_me)&set(g_s_ge)
overlap_g_s=list(overlap_g_s)
print("Length of overlap genes:")
print(len(overlap_g_s))
print("Overlapped gene symbols:")
print(overlap_g_s)